## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-26-06-34-48 +0000,nypost,LA home linked to ‘TikTok cult’ pastor seen in...,https://nypost.com/2025/07/26/us-news/californ...
1,2025-07-26-06-30-50 +0000,nypost,South Carolina dance teacher arrested after qu...,https://nypost.com/2025/07/26/us-news/south-ca...
2,2025-07-26-06-10-33 +0000,wapo,Pope Leo reflects on migrants and refugees as ...,https://www.washingtonpost.com/immigration/202...
3,2025-07-26-06-02-35 +0000,nyt,Trump’s Trip to Scotland Echoes an Earlier Vis...,https://www.nytimes.com/2025/07/26/world/europ...
4,2025-07-26-05-49-42 +0000,nyt,Taiwan Votes in Recall Campaign That Tests Chi...,https://www.nytimes.com/2025/07/25/world/asia/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2361/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
29,trump,49
79,new,19
114,gaza,17
193,will,11
287,war,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
184,2025-07-25-15-18-56 +0000,bbc,Trump says Hamas 'didn't want deal' as US and ...,https://www.bbc.com/news/articles/crrqrz857wgo,106
167,2025-07-25-16-25-00 +0000,wsj,President Trump has long wanted a U.S. soverei...,https://www.wsj.com/politics/policy/sovereign-...,92
215,2025-07-25-13-23-00 +0000,wsj,Trump Administration Sues New York City Over S...,https://www.wsj.com/us-news/trump-administrati...,91
61,2025-07-25-23-06-53 +0000,wapo,"Amid starvation scenes in Gaza, Trump administ...",https://www.washingtonpost.com/national-securi...,88
140,2025-07-25-18-31-45 +0000,nypost,Elon Musk hits back at Trump after president s...,https://nypost.com/2025/07/25/business/elon-mu...,82


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
184,106,2025-07-25-15-18-56 +0000,bbc,Trump says Hamas 'didn't want deal' as US and ...,https://www.bbc.com/news/articles/crrqrz857wgo
106,67,2025-07-25-20-45-00 +0000,wsj,Rep. Alexandria Ocasio-Cortez of New York stil...,https://www.wsj.com/politics/policy/aoc-met-ga...
167,43,2025-07-25-16-25-00 +0000,wsj,President Trump has long wanted a U.S. soverei...,https://www.wsj.com/politics/policy/sovereign-...
46,40,2025-07-26-00-25-07 +0000,nyt,Thailand Warns of War With Cambodia as Deadly ...,https://www.nytimes.com/2025/07/25/world/asia/...
97,37,2025-07-25-21-10-00 +0000,cbc,France plans to recognize Palestinian statehoo...,https://www.cbc.ca/news/politics/france-macron...
40,35,2025-07-26-01-00-00 +0000,wsj,The Democratic Party’s image has eroded to its...,https://www.wsj.com/politics/elections/democra...
24,31,2025-07-26-02-23-22 +0000,nypost,Democratic Texas congressional candidate dragg...,https://nypost.com/2025/07/25/us-news/democrat...
6,30,2025-07-26-05-28-38 +0000,wapo,Southwest flight drops 500 feet in evasive act...,https://www.washingtonpost.com/transportation/...
49,29,2025-07-26-00-09-39 +0000,nyt,Judge Dismisses Trump Administration Lawsuit A...,https://www.nytimes.com/2025/07/25/us/illinois...
124,29,2025-07-25-19-45-49 +0000,nyt,He Placated Regulators to Buy Paramount. But C...,https://www.nytimes.com/2025/07/25/business/me...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
